In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
import requests
from urllib.request import urlopen
from datetime import datetime
from tqdm import tqdm
import time
import jsonlines
import ast
import codecs
import langid

In [2]:
# Specify the path to the downloaded ChromeDriver executable
driver_path = 'chromedriver_win32/chromedriver.exe'

# Create a service object using the driver path
service = Service(driver_path)

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service)

#Go to URL
url = 'https://www.thepatriots.asia/'
driver.get(url)

#Continuously press load more button until all articles are loaded
MAX_RETRIES = 13
retry_count = 0
click_count = 0

print("Begin Load More Clicking")
while retry_count < MAX_RETRIES:
    
    notif_button = driver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div[2]/button[2]')
    if len(notif_button) > 0:
            notif_button[0].click()
            
    notif_button2 = driver.find_elements(By.XPATH, '/html/body/div[4]/div/div/div[2]/button[2]')
    if len(notif_button2) > 0:
            notif_button2[0].click()
            
    time.sleep(1.3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 100);")
    time.sleep(1.3)
    try:
        load_more_button = driver.find_element(
            By.XPATH,
            '/html/body/div[1]/div[5]/div/div/section[5]/div/div[1]/div/div/div/section/div/div[2]/a'
        )
        load_more_button.click()
        time.sleep(1.3)

        # Wait for a specific element to be present
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located(
            (
                By.XPATH,
                '/html/body/div[1]/div[5]/div/div/section[5]/div/div[1]/div/div/div/section/div/div[2]/a'
            )))

        # If the element is found, reset the retry_count and continue with further actions
        retry_count = 0
        click_count += 1

    except Exception as e:
        retry_count += 1
        print(f"Load More button not found. Retry: {retry_count}/{MAX_RETRIES}")

# Check if the maximum number of retries has been reached
if retry_count == MAX_RETRIES:
    print("Reached maximum retries. Unable to find Load More button.")
    print(f"Clicked {click_count} times")

Begin Load More Clicking
Load More button not found. Retry: 1/13
Load More button not found. Retry: 2/13
Load More button not found. Retry: 3/13
Load More button not found. Retry: 4/13
Load More button not found. Retry: 5/13
Load More button not found. Retry: 6/13
Load More button not found. Retry: 7/13
Load More button not found. Retry: 8/13
Load More button not found. Retry: 9/13
Load More button not found. Retry: 10/13
Load More button not found. Retry: 11/13
Load More button not found. Retry: 12/13
Load More button not found. Retry: 13/13
Reached maximum retries. Unable to find Load More button.
Clicked 760 times


In [24]:
patriots_hrefs = []
# Find all div with class "block-content"
div_elements = driver.find_elements(By.CLASS_NAME, 'block-content')

for div_element in div_elements:
    # Find all anchor tags within the div element
    anchor_tags = div_element.find_elements(By.TAG_NAME, 'a')

    # Extract the href attribute from each anchor tag
    href_list = [tag.get_attribute('href') for tag in anchor_tags]

    # Print the extracted href values
    for href in href_list:
        if "/author/" not in href and "/category/" not in href: #get only article link
            patriots_hrefs.append(href)

In [26]:
len(patriots_hrefs)

21178

In [31]:
set_patriots_hrefs = list(set(patriots_hrefs))

In [32]:
len(set_patriots_hrefs)

10592

In [34]:
set_patriots_hrefs[0]

'https://www.thepatriots.asia/merungkai-kemisterian-tulisan-helikopter-di-kuil-firaun/'

In [ ]:
for href in tqdm(set_patriots_hrefs):
    while True:
        try:
            r = requests.get(href)
            if r.status_code == 200:
                break
            else:
                print(r.status_code)
        except Exception as e:
            print(e)
            time.sleep(3.9)
            
    soup = BeautifulSoup(r.text, "lxml")
    title = soup.find("h1", attrs = {"class":"is-title post-title au j"}).text.strip()

    div = soup.find_all('div', attrs = {"class":"post-content-wrap has-share-float ttu"})
    p = [p.text for p in div[0].find_all(["p"]) if len(p.text.split()) > 0]
    cleaned_p = []

    append_into = True
    for text in p:
        if "Perhatian sebentar…" in text: #remove text asking for donation
            append_into = False

        if append_into:
            cleaned_p.append(text)

        if "RM2 / RM5 / RM10 / RM50" in text:
            append_into = True
    
    data = {'url': href, 'headline': title, 'content': cleaned_p}
    
    with open('patriots-before.jsonl', 'a') as f:
        json.dump(data, f)
        f.write('\n')

In [ ]:
# Specify the input and output file paths
input_file = "patriots-before.jsonl"
output_file = "patriots.jsonl"

langid.set_languages(['ms', 'en'])  #ISO 639-1 codes

# Open the input and output files
with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
    # Iterate over each line in the input file
    for line in reader:
        # Check if the content field is empty ([] or {})
        if line['content']:
            # Write the non-empty line to the output file
            decoded_list = []
            for string in line['content']:
                try:
                    decoded_string = codecs.escape_decode(string)[0].decode('utf-8')
                except ValueError:
                    decoded_string = string
                decoded_list.append(decoded_string)
            line['content'] = decoded_list

            writer.write(line)